En este docuemnto realizamos la carga de los datos y realizamos un análisis general de los distintos años. Revisaremos que todas las columnas contengan los datos correctos, que los tipos de dato sean correctos, realizaremos un tratamiento de los valores nulos y de los duplicados. Finalmente realizaremos la unión de todos los datos.

In [1]:
# Importamos las librerías a utilizar
import pandas as pd
import numpy as np

In [2]:
# Indicamos que queremos que nos enseñe por pantalla todas las columnas de los DataFrame que vamos a cargar
pd.set_option('display.max_columns', None)

Ponemos las funciones que usaremos en el análisis

In [3]:
def info_df(df):
    """
    Función que devuelve información general sobre el DatFrame que le pasemos.


    Args:
        df (DataFrame): DataFrame con información que queramos revisar

    Returns:
        DataFrame: DataFrame con información general sobre las columnas del DataFrame que se le ha pasado a la función: tipo de datos, número de
        registros, número de valores nulos, porcentaje de los valores nulos sobre el total
    """
    info_df = pd.DataFrame()
    info_df["Tipo_dato"] = df.dtypes
    info_df["numero_registros"] = [df[elemento].value_counts().sum() for elemento in df]
    info_df["Numero_nulos"] = round(df.isnull().sum())
    info_df["%_nulos"] = round((df.isnull().sum()/df.shape[0])*100, 2)

    return info_df

# Análisis año a año del desarrollo económico de Brasil

**Datos 2013**

In [4]:
# Nos ha surgido un problema a la hora de cargar los datos debido a que los datos no están en español, sino en portugués (son datos de Brasil).
# Para arreglar este problema aplicamos el encoding `latin-1`
datos_2013 = pd.read_csv("datos/datos-2013.csv", sep = ";", encoding = "latin-1")
datos_2013. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",31/12/2013,2013


In [5]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2013 = datos_2013.copy()
copia_datos_2013.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",31/12/2013,2013


Información general del DataFrame de 2013

In [7]:
# Realizamos una revisión de la información general del dataframe de los datos de 2013.
print(datos_2013.shape)
info_df(datos_2013)

(4498, 16)


,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4498,0,0.0
NOME ÓRGÃO SUPERIOR,object,4498,0,0.0
CÓDIGO ÓRGÃO,int64,4498,0,0.0
NOME ÓRGÃO,object,4498,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4498,0,0.0
NOME UNIDADE GESTORA,object,4498,0,0.0
CATEGORIA ECONÔMICA,object,4498,0,0.0
ORIGEM RECEITA,object,4498,0,0.0
ESPÉCIE RECEITA,object,4498,0,0.0
DETALHAMENTO,object,4498,0,0.0


In [5]:
# Nos encontramos con varias columnas que contienen datos númericos que tienen formato objeto. 

print(datos_2013["VALOR PREVISTO ATUALIZADO"].dtypes)
print(datos_2013["VALOR LANÇADO"].dtypes)
print(datos_2013["VALOR REALIZADO"].dtypes)
print(datos_2013["PERCENTUAL REALIZADO"].dtypes)

# Para poder trabajar con estas columnas vamos a necesitar cambiar el tipo de dato de las mismas.

object
object
object
object


In [6]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2013 = datos_2013.iloc[:,10: 14].columns
columnas_2013

Index(['VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO', 'VALOR REALIZADO',
       'PERCENTUAL REALIZADO'],
      dtype='object')

In [7]:
# Para poder realizar el cambio de estas columnas del tipo de dato, anteriormente necesitaremos cambiar las comas por puntos, ya que python
# no reconoce las comas en los números, solo los puntos. Para ello aplicaremos un replace sobre cada uno de los elementos de las columnas, 
# y pediremos que nos modifique las comas por puntos.
def buscar_reemplazar(dataframe, columnas):

    """
    Función que cambia los datos numéricos de una columna de un dataframe, que están en tipo string, y contienen comas. 

    Args:
        dataframe (DataFrame): Dataframe que contiene las columnas a modificar
        columnas (lista de columnas del DataFrame): lista de columnas del DataFrame, de las cuales se quiere realizar el cambio de datos.

    Returns:
        DataFrame: dataframe con las columnas indicadas con los datos numéricos cambiados a tipo float.
    """
    for columna in columnas:
    
        dataframe[columna] = dataframe[columna].str.replace(",", ".").astype(float)

    return dataframe

In [8]:
# Aplicamos la función y revisamos que el cambio se ha realizado con éxito.
datos_2013 = buscar_reemplazar(datos_2013, columnas_2013)
datos_2013.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,0.0,0.0,1297.13,0.0,31/12/2013,2013


In [9]:
# En las siguientes columnas tenemos información de tipo fecha pero las tenemos en tipo objeto o entero. Vamos a realizar el tipo de cambio de dato
# de la columna que tenemos en tipo objeto, ya que la de tipo entero no merece la pena realizar el cambio.
print(datos_2013["DATA LANÇAMENTO"].dtypes)
print(datos_2013["ANO EXERCÍCIO"].dtypes)

object
int64


In [10]:
datos_2013["DATA LANÇAMENTO"] = pd.to_datetime(datos_2013["DATA LANÇAMENTO"], format="%d/%m/%Y")
datos_2013.head(2)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,0.0,0.0,1297.13,0.0,2013-12-31,2013
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,0.0,0.0,26666621.42,0.0,2013-12-31,2013


In [13]:
# Comprobamos que el cambio de tipo de dato se ha realizado de forma correcta.
datos_2013["DATA LANÇAMENTO"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4498 entries, 0 to 4497
Series name: DATA LANÇAMENTO
Non-Null Count  Dtype         
--------------  -----         
4498 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 35.3 KB


In [15]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2013_fecha = datos_2013.iloc[:,14]
columnas_2013_fecha

0       31/12/2013
1       31/12/2013
2       31/12/2013
3       31/12/2013
4       31/12/2013
           ...    
4493    31/12/2013
4494    31/12/2013
4495    31/12/2013
4496    31/12/2013
4497    31/12/2013
Name: DATA LANÇAMENTO, Length: 4498, dtype: object

In [16]:
# Para poder realizar el cambio de dato de str a datetime nos creamos la siguiente función
def cambio_fecha(dataframe, fechas):
    """Función que recibe una lista de columnas de un DataFrame y les cambia el tipo de dato a datetime

    Args:
        dataframe (DataFrame): DataFrame
        lista_columnas (columnas del DataFrame): columnas del DataFrame pasado a la función. 

    Returns:
        DataFrame: devolverá un valor nulo (np.nan) en caso de que se encuentre con un valor nulo,
        o el valor en tipo datetime en el resto de casos. 
    """
    for fecha in fechas:
        if fecha == None:
            return np.nan
        else:
            return pd.to_datetime(fecha)
    

In [41]:
datos_2013 = cambio_fecha(datos_2013, columnas_2013_fecha)
datos_2013.head()

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,0.0,0.0,1297.13,0.0,2013-12-31,1970-01-01 00:00:00.000002013
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,0.0,0.0,26666621.42,0.0,2013-12-31,1970-01-01 00:00:00.000002013
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,0.0,0.0,301251.13,0.0,2013-12-31,1970-01-01 00:00:00.000002013
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIV.ATIVA POR INFRAÇÃO ADMINISTRATIVA,0.0,0.0,1855.58,0.0,2013-12-31,1970-01-01 00:00:00.000002013
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",OUTRAS RESTITUICOES,0.0,0.0,52140.68,0.0,2013-12-31,1970-01-01 00:00:00.000002013


In [15]:
# Volvemos a revisar la información general del dataframe tras haber realizado los cambios para ver que todo ha sido realizado de forma correcta.
info_df(datos_2013)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4498,0,0.0
NOME ÓRGÃO SUPERIOR,object,4498,0,0.0
CÓDIGO ÓRGÃO,int64,4498,0,0.0
NOME ÓRGÃO,object,4498,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4498,0,0.0
NOME UNIDADE GESTORA,object,4498,0,0.0
CATEGORIA ECONÔMICA,object,4498,0,0.0
ORIGEM RECEITA,object,4498,0,0.0
ESPÉCIE RECEITA,object,4498,0,0.0
DETALHAMENTO,object,4498,0,0.0


In [16]:
# Revisamos si existen valores duplicados en el dataframe
numero_duplicados_df_2013 = datos_2013.duplicated().sum()
numero_duplicados_df_2013

np.int64(0)

Información general sobre el dataframe de 2013:
- Todas las columnas contienen 4.498 entradas. 
- No existen entradas con valor nulo en ninguna de las columnas.
- No existen valores duplicados en el dataframe.
- Nos encontramos con los siguientes tipos de datos: object, float, int, datetime. 

A continuación realizamos una revisión de los valores que nos encontramos en cada columna.

In [33]:
# Valores de la columna de `Organización superior`

print(f"Las organizaciones superiores que nos encontramos en la adminsitración de Brasil en 2013 son: \n \n { datos_2013["NOME ÓRGÃO SUPERIOR"].unique()}")


Las organizaciones superiores que nos encontramos en la adminsitración de Brasil en 2013 son: 
 
 ['Advocacia-Geral da União' 'Controladoria-Geral da União'
 'Ministério da Agricultura, Pecuária e Abastec' 'Ministério da Cidadania'
 'Ministério da Ciência, Tecnologia, Inovações ' 'Ministério da Defesa'
 'Ministério da Economia' 'Ministério da Educação'
 'Ministério da Infraestrutura'
 'Ministério da Justiça e Segurança Pública'
 'Ministério da Mulher, Família e Direitos Huma'
 'Ministério da Pesca e Aquicultura' 'Ministério da Previdência Social'
 'Ministério da Saúde' 'Ministério das Comunicações'
 'Ministério das Mulheres, Igualdade Racial, da'
 'Ministério das Relações Exteriores' 'Ministério de Minas e Energia'
 'Ministério do Desenvolvimento Agrário'
 'Ministério do Desenvolvimento Regional' 'Ministério do Esporte'
 'Ministério do Meio Ambiente' 'Ministério do Trabalho e Emprego'
 'Ministério do Turismo' 'Presidência da República']


In [35]:
# Valores de la columna de `Nombre de organización`.

print(f"Las entidades gubernamentales específicas que nos encontramos en la adminsitración de Brasil en 2013 son: \n \n { datos_2013["NOME ÓRGÃO"].unique()}")

Las entidades gubernamentales específicas que nos encontramos en la adminsitración de Brasil en 2013 son: 
 
 ['Advocacia-Geral da União - Unidades com vínculo direto'
 'Controladoria-Geral da União'
 'Empresa Brasileira de Pesquisa Agropecuária'
 'Ministério da Agricultura, Pecuária e Abastecimento - Unidades com vínculo direto'
 'Companhia Nacional de Abastecimento' 'Serviço Florestal Brasileiro'
 'Instituto Nacional de Colonização e Reforma Agrária'
 'Fundo de Defesa da Economia Cafeeira'
 'Fundo de Terras e da Reforma Agrária'
 'Ministério da Cidadania - Unidades com vínculo direto'
 'Fundo Nacional de Assistência Social'
 'Ministério da Ciência, Tecnologia, Inovações e Comunicações - Unidades com vínculo direto'
 'Conselho Nacional de Desenvolvimento Científico e Tecnológico'
 'Comissão Nacional de Energia Nuclear'
 'Fundo Nacional de Desenvolvimento Científico e Tecnológico'
 'Financiadora de Estudos e Projetos'
 'Centro Nacional de Tecnologia Eletrônica Avançada S.A.'
 'Agência 

In [36]:
# Valores de la columna de `Nombre de organización gestora`.

print(f"Las organizaciones gestoras que nos encontramos en la adminsitración de Brasil en 2013 son: \n \n { datos_2013["NOME UNIDADE GESTORA"].unique()}")

Las organizaciones gestoras que nos encontramos en la adminsitración de Brasil en 2013 son: 
 
 ['COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU'
 'DIRETORIA DE GESTAO INTERNA/SE/CGU' 'EMBRAPA-SETORIAL'
 'COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA'
 'COMPANHIA NACIONAL DE ABASTECIMENTO'
 'SETORIAL PROG.ORCAMENTARIA E FINANCEIRA-SFB'
 'DPTO. DE ADMINISTRACAO FINANCEIRA-DAF - INCRA'
 'FUNDO DE DEFESA DA ECON.CAFEEIRA/FUNCAFE/SPA' 'CREDITO FUNDIARIO'
 'COORD.GERAL DE PLAN. ORC. FIN. E CONTAB./MINC'
 'DEPARTAMENTO DE GESTAO INTERNA' 'SETORIAL DE ORCAMENTO E FINANCAS/MC'
 'FUNDO NACIONAL DE ASSISTENCIA SOCIAL'
 'COORDENACAO-GERAL DE ORCAMENTO E FINANCAS'
 'CNPQ - ADMINISTRACAO CENTRAL' 'CNEN-ORCAMENTO E FINANCAS'
 'FUNDO NAC.DE DESENV. CIENT. E TECNOLOGICO'
 'FINANCIADORA DE ESTUDOS E PROJETOS'
 'CENTRO NACIONAL DE TECNOLOGIA ELETRONICA AVAN'
 'CNPQ - DIFERENCIA CAMBIAL' 'AGENCIA ESPACIAL BRASILEIRA/AEB'
 'EMPRESA BRASILEIRA DE CORREIOS E TELEGRAFOS'
 'SECRETARIA DE ECON. E FIN.DA AER.- F.AER

In [64]:
# Valores de la columna de `Categoría económica`.

print("Los ingresos de Brasil se clasifican en las siguientes categorías en 2013: \n ")
detalle_categoría_economica = pd.DataFrame(datos_2013["CATEGORIA ECONÔMICA"].value_counts())
detalle_categoría_economica["Peso categoría"] = round(((detalle_categoría_economica["count"]/datos_2013.shape[0])*100), 2)
detalle_categoría_economica

Los ingresos de Brasil se clasifican en las siguientes categorías en 2013: 
 


,count,Peso categoría
CATEGORIA ECONÔMICA,,
Receitas Correntes,4001,88.95
Receitas de Capital,259,5.76
Receitas Correntes - intra-orçamentárias,236,5.25
Sem informação,1,0.02
Receitas de Capital - intra-orçamentárias,1,0.02


In [58]:
# Valores de la columna de `Origen de ingresos`.

print("Los ingresos de Brasil provienen de las siguientes categorías en 2013: \n ")
detalle_origen_ingreso = pd.DataFrame(datos_2013["ORIGEM RECEITA"].value_counts())
detalle_origen_ingreso


Los ingresos de Brasil provienen de las siguientes categorías en 2013: 
 


,count
ORIGEM RECEITA,
Outras Receitas Correntes,1982
Receita de Serviços,978
Receita Patrimonial,621
Alienação de Bens,184
"Impostos, Taxas e Contribuições de Melhoria",176
Contribuições,176
Transferências Correntes,126
Receita Agropecuária,105
Receita Industrial,61


In [57]:
# Valores de la columna de `Especie de ingreso`.

print("Los ingresos de Brasil provienen de las siguientes categorías en 2013: \n ")
detalle_especie_ingreso = pd.DataFrame(datos_2013["ESPÉCIE RECEITA"].value_counts())
detalle_especie_ingreso

Los ingresos de Brasil provienen de las siguientes categorías en 2013: 
 


,count
ESPÉCIE RECEITA,
Receita de Serviços,978
"Multas administrativas, contratuais e judicia",911
"Indenizações, restituições e ressarcimentos",654
Exploração do patrimônio imobiliário do Estad,302
Demais receitas correntes,229
"Bens, Direitos e Valores Incorporados ao Patr",163
Valores Mobiliários,160
Alienação de bens móveis,156
Impostos,124


In [54]:
# Valores de la columna de `Detalle adicional del tipo de ingreso`.
print("Los ingresos de Brasil provienen de las siguientes categorías en 2013: \n")
detalle_adicional_ingreso = pd.DataFrame(datos_2013["DETALHAMENTO"].value_counts())
detalle_adicional_ingreso

Los ingresos de Brasil provienen de las siguientes categorías en 2013: 



,count
DETALHAMENTO,
RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,214
SERVICOS ADMINISTRATIVOS,195
OUTRAS RESTITUICOES,173
OUTRAS RECEITAS,162
ALUGUEIS,147
...,...
REC.DIV.ATIV.SERVIC.INSP.FISC.ATIV.MINERAL,1
REC.DIV.ATIV.-MULTA PREV. LEG. LUB. E COMBUST,1
ALIEN. BENS IMOVEIS DE DOM DA UNIAO-DOM PLENO,1


In [44]:
# Valores de la columna de `Detalle adicional del tipo de ingreso`.

print(f"Los ingresos de Brasil han sido registrados en las siguientes fechas en 2013: \n \n { datos_2013["DATA LANÇAMENTO"].unique()}")

Los ingresos de Brasil han sido registrados en las siguientes fechas en 2013: 
 
 <DatetimeArray>
['2013-12-31 00:00:00']
Length: 1, dtype: datetime64[ns]


In [48]:
datos_2013["VALOR PREVISTO ATUALIZADO"].value_counts()

VALOR PREVISTO ATUALIZADO
0.0          1625
6000000.0       5
300000.0        3
60000.0         3
1102.0          3
             ... 
5760731.0       1
20149.0         1
6379021.0       1
123985.0        1
447630.0        1
Name: count, Length: 2788, dtype: int64

In [45]:
# Valores de la columna de `Detalle adicional del tipo de ingreso`.

print(f"Los ingresos de Brasil han sido registrados en las siguientes fechas en 2013: \n \n { datos_2013["ANO EXERCÍCIO"].unique()}")

Los ingresos de Brasil han sido registrados en las siguientes fechas en 2013: 
 
 [2013]


Las columnas del dataframe de 2013 nos proporcionan información sobre como se estructura la contabilidad, la balanza de pagos de Brasil en este año. Realizaremos un estudio más profundo sobre su estructura contable, y la proveniencia de los ingresos de este país.

In [ ]:
# Guardamos los datos de 2013 en formato pickle para poder trabajar con ellos en otros documentos con las modificaciones que hemos aplicado
# en este documento.
pd.to_pickle(datos_2013, "datos/datos_2013.pkl")


**Datos 2014**

In [65]:
# Importamos los datos del año 2014
datos_2014 = pd.read_csv("datos/datos-2014.csv", sep = ";", encoding = "latin-1")
datos_2014. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,"0,00","0,00","46699948,08","0,00",31/12/2014,2014


In [66]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2014 = datos_2014.copy()
copia_datos_2014.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,"0,00","0,00","46699948,08","0,00",31/12/2014,2014


Información general del DataFrame de 2014

In [68]:
# Realizamos una revisión de la información general del dataframe de los datos de 2014.
print(datos_2014.shape)
info_df(datos_2014)

(4553, 16)


,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4553,0,0.0
NOME ÓRGÃO SUPERIOR,object,4553,0,0.0
CÓDIGO ÓRGÃO,int64,4553,0,0.0
NOME ÓRGÃO,object,4553,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4553,0,0.0
NOME UNIDADE GESTORA,object,4553,0,0.0
CATEGORIA ECONÔMICA,object,4553,0,0.0
ORIGEM RECEITA,object,4553,0,0.0
ESPÉCIE RECEITA,object,4553,0,0.0
DETALHAMENTO,object,4553,0,0.0


No hay valores nulos en el DataFrame de 2014

In [69]:
# Nos encontramos con varias columnas que contienen datos númericos que tienen formato objeto.
 
print(datos_2014["VALOR PREVISTO ATUALIZADO"].dtypes)
print(datos_2014["VALOR LANÇADO"].dtypes)
print(datos_2014["VALOR REALIZADO"].dtypes)
print(datos_2014["PERCENTUAL REALIZADO"].dtypes)

# Para poder trabajar con estas columnas vamos a necesitar cambiar el tipo de dato de las mismas.

object
object
object
object


In [70]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2014 = datos_2014.iloc[:,10: 14].columns
columnas_2014

Index(['VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO', 'VALOR REALIZADO',
       'PERCENTUAL REALIZADO'],
      dtype='object')

In [71]:
# Aplicamos la función de cambio de tipo de datos y revisamos que el cambio se ha realizado con éxito.
datos_2014 = buscar_reemplazar(datos_2014, columnas_2014)
datos_2014.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,0.0,0.0,46699948.08,0.0,31/12/2014,2014


In [72]:
# En las siguientes columnas tenemos información de tipo fecha pero las tenemos en tipo objeto o entero. Vamos a realizar el tipo de cambio de dato
# de la columna que tenemos en tipo objeto, ya que la de tipo entero no merece la pena realizar el cambio.
print(datos_2014["DATA LANÇAMENTO"].dtypes)
print(datos_2014["ANO EXERCÍCIO"].dtypes)

object
int64


In [73]:
datos_2014["DATA LANÇAMENTO"] = pd.to_datetime(datos_2014["DATA LANÇAMENTO"], format="%d/%m/%Y")
datos_2014.head(2)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORÁRIOS DE ADVOGADOS,0.0,0.0,46699948.08,0.0,2014-12-31,2014
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",MULTAS E JUROS PREVISTOS EM CONTRATOS,0.0,0.0,15920.83,0.0,2014-12-31,2014


In [77]:
# Comprobamos que el cambio de tipo de dato se ha realizado de forma correcta.
datos_2014["DATA LANÇAMENTO"].dtypes
# Aunque nos devuelva en M8 está bien, pendiente explicación del M8. 

dtype('<M8[ns]')

In [78]:
# Revisamos si existen valores duplicados en el dataframe
numero_duplicados_df_2014 = datos_2014.duplicated().sum()
numero_duplicados_df_2014

np.int64(0)

Información general sobre el dataframe de 2014:
- Todas las columnas contienen 4.553 entradas. 
- No existen entradas con valor nulo en ninguna de las columnas.
- No existen valores duplicados en el dataframe.
- Nos encontramos con los siguientes tipos de datos: object, float, int, datetime. 

**Datos 2015**

In [79]:
# Importamos los datos del año 2015
datos_2015 = pd.read_csv("datos/datos-2015.csv", sep = ";", encoding = "latin-1")
datos_2015. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORARIOS DE ADVOGADOS,"0,00","0,00","63829853,40","0,00",31/12/2015,2015


In [80]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2015 = datos_2015.copy()
copia_datos_2015.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORARIOS DE ADVOGADOS,"0,00","0,00","63829853,40","0,00",31/12/2015,2015


Información general del DataFrame de 2014

In [81]:
# Realizamos una revisión de la información general del dataframe de los datos de 2014.
print(datos_2015.shape)
info_df(datos_2015)

(4523, 16)


,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,4523,0,0.0
NOME ÓRGÃO SUPERIOR,object,4523,0,0.0
CÓDIGO ÓRGÃO,int64,4523,0,0.0
NOME ÓRGÃO,object,4523,0,0.0
CÓDIGO UNIDADE GESTORA,int64,4523,0,0.0
NOME UNIDADE GESTORA,object,4523,0,0.0
CATEGORIA ECONÔMICA,object,4523,0,0.0
ORIGEM RECEITA,object,4523,0,0.0
ESPÉCIE RECEITA,object,4523,0,0.0
DETALHAMENTO,object,4523,0,0.0


In [83]:
# Los datos de la columna del monto previsto actualizado contiene números que tienen formato objeto, cuando son datos numéricos. 
print(datos_2015["VALOR PREVISTO ATUALIZADO"].dtypes)
print(datos_2015["VALOR LANÇADO"].dtypes)
print(datos_2015["VALOR REALIZADO"].dtypes)
print(datos_2015["PERCENTUAL REALIZADO"].dtypes)
# Para poder trabajar con estas columnas vamos a necesitar cambiar el tipo de dato de las mismas.

object
object
object
object


In [84]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2015 = datos_2015.iloc[:,10: 14].columns
columnas_2015

Index(['VALOR PREVISTO ATUALIZADO', 'VALOR LANÇADO', 'VALOR REALIZADO',
       'PERCENTUAL REALIZADO'],
      dtype='object')

In [85]:
# Aplicamos la función de cambio de tipo de datos y revisamos que el cambio se ha realizado con éxito.
datos_2015 = buscar_reemplazar(datos_2015, columnas_2015)
datos_2015.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORARIOS DE ADVOGADOS,0.0,0.0,63829853.4,0.0,31/12/2015,2015


In [87]:
# En las siguientes columnas tenemos información de tipo fecha pero las tenemos en tipo objeto o entero. Vamos a realizar el tipo de cambio de dato
# de la columna que tenemos en tipo objeto, ya que la de tipo entero no merece la pena realizar el cambio.
print(datos_2015["DATA LANÇAMENTO"].dtypes)
print(datos_2015["ANO EXERCÍCIO"].dtypes)

object
int64


In [89]:
datos_2015["DATA LANÇAMENTO"] = pd.to_datetime(datos_2015["DATA LANÇAMENTO"], format="%d/%m/%Y")
datos_2015.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,RECEITA DE HONORARIOS DE ADVOGADOS,0.0,0.0,63829853.4,0.0,2015-12-31,2015


In [90]:
# Comprobamos que el cambio de tipo de dato se ha realizado de forma correcta.
datos_2014["DATA LANÇAMENTO"].dtypes
# Aunque nos devuelva en M8 está bien, pendiente explicación del M8.

dtype('<M8[ns]')

In [91]:
# Revisamos si existen valores duplicados en el dataframe
numero_duplicados_df_2015 = datos_2015.duplicated().sum()
numero_duplicados_df_2015

np.int64(0)

Información general sobre el dataframe de 2015:
- Todas las columnas contienen 4.523 entradas. 
- No existen entradas con valor nulo en ninguna de las columnas.
- No existen valores duplicados en el dataframe.
- Nos encontramos con los siguientes tipos de datos: object, float, int, datetime. 

**Datos 2016**

In [59]:
# Importamos los datos del año 2016
datos_2016 = pd.read_csv("datos/datos-2016.csv", sep = ";", encoding = "latin-1")
datos_2016. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ENCARGOS LEGAIS PELA INSCR.EM DIV.ATIVA-PRINC,"0,00","0,00","154948,98","0,00",04/04/2016,2016


In [60]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2016 = datos_2016.copy()
copia_datos_2016.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ENCARGOS LEGAIS PELA INSCR.EM DIV.ATIVA-PRINC,"0,00","0,00","154948,98","0,00",04/04/2016,2016


Información general sobre los datos de 2016.

In [61]:
info_df(datos_2016)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,194533,0,0.00
NOME ÓRGÃO SUPERIOR,object,194533,0,0.00
CÓDIGO ÓRGÃO,int64,194533,0,0.00
NOME ÓRGÃO,object,194533,0,0.00
CÓDIGO UNIDADE GESTORA,int64,194533,0,0.00
NOME UNIDADE GESTORA,object,194533,0,0.00
CATEGORIA ECONÔMICA,object,194533,0,0.00
ORIGEM RECEITA,object,194533,0,0.00
ESPÉCIE RECEITA,object,194533,0,0.00
DETALHAMENTO,object,194533,0,0.00


Hay valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [62]:
numero_duplicados_df_2016 = datos_2016.duplicated().sum()
numero_duplicados_df_2016

np.int64(0)

In [63]:
# Como queremos modificar las columnas anteriores, nos creamos una lista que contenga las mismas. Para ello hacemos uso de iloc. 
columnas_2016_fecha = datos_2016.iloc[:,14: 16].columns
columnas_2016_fecha

Index(['DATA LANÇAMENTO', 'ANO EXERCÍCIO'], dtype='object')

In [64]:
datos_2016 = cambio_fecha(datos_2016, columnas_2016_fecha)
datos_2016.head(2)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ENCARGOS LEGAIS PELA INSCR.EM DIV.ATIVA-PRINC,"0,00","0,00","154948,98","0,00",2016-04-04,1970-01-01 00:00:00.000002016
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,ONUS DE SUCUMBENCIA-MULTAS E JUROS,"0,00","0,00","80,84","0,00",2016-08-12,1970-01-01 00:00:00.000002016


In [65]:
info_df(datos_2016)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,194533,0,0.00
NOME ÓRGÃO SUPERIOR,object,194533,0,0.00
CÓDIGO ÓRGÃO,int64,194533,0,0.00
NOME ÓRGÃO,object,194533,0,0.00
CÓDIGO UNIDADE GESTORA,int64,194533,0,0.00
NOME UNIDADE GESTORA,object,194533,0,0.00
CATEGORIA ECONÔMICA,object,194533,0,0.00
ORIGEM RECEITA,object,194533,0,0.00
ESPÉCIE RECEITA,object,194533,0,0.00
DETALHAMENTO,object,194533,0,0.00


**Datos 2017**

In [66]:
# Importamos los datos del año 2017
datos_2017 = pd.read_csv("datos/datos-2017.csv", sep = ";", encoding = "latin-1")
datos_2017. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,OUTRAS RECEITAS-PRIMARIAS-PRINCIPAL,"0,00","0,00","198,00","0,00",29/08/2017,2017


In [67]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2017 = datos_2017.copy()
copia_datos_2017.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,OUTRAS RECEITAS-PRIMARIAS-PRINCIPAL,"0,00","0,00","198,00","0,00",29/08/2017,2017


In [68]:
info_df(datos_2017)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,190479,0,0.00
NOME ÓRGÃO SUPERIOR,object,190479,0,0.00
CÓDIGO ÓRGÃO,int64,190479,0,0.00
NOME ÓRGÃO,object,190479,0,0.00
CÓDIGO UNIDADE GESTORA,int64,190479,0,0.00
NOME UNIDADE GESTORA,object,190479,0,0.00
CATEGORIA ECONÔMICA,object,190479,0,0.00
ORIGEM RECEITA,object,190479,0,0.00
ESPÉCIE RECEITA,object,190479,0,0.00
DETALHAMENTO,object,190479,0,0.00


Hay 86 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [69]:
numero_duplicados_df_2017 = datos_2017.duplicated().sum()
numero_duplicados_df_2017

np.int64(0)

**Datos 2018**

In [70]:
# Importamos los datos del año 2018
datos_2018 = pd.read_csv("datos/datos-2018.csv", sep = ";", encoding = "latin-1")
datos_2018. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receitas Correntes - a classificar,Receitas Correntes - a classificar,Receitas Correntes - a classificar,"0,00","0,00","-1693,72","0,00",25/07/2018,2018


In [71]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2018 = datos_2018.copy()
copia_datos_2018.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receitas Correntes - a classificar,Receitas Correntes - a classificar,Receitas Correntes - a classificar,"0,00","0,00","-1693,72","0,00",25/07/2018,2018


In [72]:
info_df(datos_2018)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,173944,0,0.00
NOME ÓRGÃO SUPERIOR,object,173944,0,0.00
CÓDIGO ÓRGÃO,int64,173944,0,0.00
NOME ÓRGÃO,object,173944,0,0.00
CÓDIGO UNIDADE GESTORA,int64,173944,0,0.00
NOME UNIDADE GESTORA,object,173944,0,0.00
CATEGORIA ECONÔMICA,object,173944,0,0.00
ORIGEM RECEITA,object,173944,0,0.00
ESPÉCIE RECEITA,object,173944,0,0.00
DETALHAMENTO,object,173944,0,0.00


Hay 115 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [73]:
numero_duplicados_df_2018 = datos_2018.duplicated().sum()
numero_duplicados_df_2018

np.int64(0)

**Datos 2019**

In [74]:
# Importamos los datos del año 2019
datos_2019 = pd.read_csv("datos/datos-2019.csv", sep = ";", encoding = "latin-1")
datos_2019. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,"0,00","0,00","-95,00","0,00",12/06/2019,2019


In [75]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2019 = datos_2019.copy()
copia_datos_2019.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Receita de Serviços,Serviços Administrativos e Comerciais Gerais,INSCR.EM CONCURSOS E PROC.SELETIVOS-PRINCIPAL,"0,00","0,00","-95,00","0,00",12/06/2019,2019


In [76]:
info_df(datos_2019)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,176828,0,0.00
NOME ÓRGÃO SUPERIOR,object,176828,0,0.00
CÓDIGO ÓRGÃO,int64,176828,0,0.00
NOME ÓRGÃO,object,176828,0,0.00
CÓDIGO UNIDADE GESTORA,int64,176828,0,0.00
NOME UNIDADE GESTORA,object,176828,0,0.00
CATEGORIA ECONÔMICA,object,176828,0,0.00
ORIGEM RECEITA,object,176828,0,0.00
ESPÉCIE RECEITA,object,176828,0,0.00
DETALHAMENTO,object,176828,0,0.00


Hay 86 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [77]:
numero_duplicados_df_2019 = datos_2019.duplicated().sum()
numero_duplicados_df_2019

np.int64(0)

**Datos 2020**

In [78]:
# Importamos los datos del año 2019
datos_2020 = pd.read_csv("datos/datos-2020.csv", sep = ";", encoding = "latin-1")
datos_2020. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RESTITUIÇÃO DE CONVÊNIOS-PRIMÁRIAS-DÍV.ATIVA,"0,00","0,00","5516,90","0,00",28/04/2020,2020


In [79]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2020 = datos_2020.copy()
copia_datos_2020.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RESTITUIÇÃO DE CONVÊNIOS-PRIMÁRIAS-DÍV.ATIVA,"0,00","0,00","5516,90","0,00",28/04/2020,2020


In [80]:
info_df(datos_2020)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,142348,0,0.00
NOME ÓRGÃO SUPERIOR,object,142348,0,0.00
CÓDIGO ÓRGÃO,int64,142348,0,0.00
NOME ÓRGÃO,object,142348,0,0.00
CÓDIGO UNIDADE GESTORA,int64,142348,0,0.00
NOME UNIDADE GESTORA,object,142348,0,0.00
CATEGORIA ECONÔMICA,object,142348,0,0.00
ORIGEM RECEITA,object,142348,0,0.00
ESPÉCIE RECEITA,object,142348,0,0.00
DETALHAMENTO,object,142348,0,0.00


Hay 86 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [81]:
numero_duplicados_df_2020 = datos_2020.duplicated().sum()
numero_duplicados_df_2020

np.int64(0)

**Datos 2021**

In [82]:
# Importamos los datos del año 2019
datos_2021 = pd.read_csv("datos/datos-2021.csv", sep = ";", encoding = "latin-1")
datos_2021. head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas de Capital,Operações de Crédito,Operações de crédito - mercado interno,TITULOS DE RESPONS.TES.NAC.-MERC.INT.-PRINC.,"2224034904,00","0,00","0,00","0,00",23/04/2021,2021


In [83]:
# Nos creamos una copia del DataFrame por si acaso
copia_datos_2021 = datos_2021.copy()
copia_datos_2021.head(1)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,CATEGORIA ECONÔMICA,ORIGEM RECEITA,ESPÉCIE RECEITA,DETALHAMENTO,VALOR PREVISTO ATUALIZADO,VALOR LANÇADO,VALOR REALIZADO,PERCENTUAL REALIZADO,DATA LANÇAMENTO,ANO EXERCÍCIO
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110060,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas de Capital,Operações de Crédito,Operações de crédito - mercado interno,TITULOS DE RESPONS.TES.NAC.-MERC.INT.-PRINC.,"2224034904,00","0,00","0,00","0,00",23/04/2021,2021


In [84]:
info_df(datos_2021)

,Tipo_dato,numero_registros,Numero_nulos,%_nulos
CÓDIGO ÓRGÃO SUPERIOR,int64,134593,0,0.00
NOME ÓRGÃO SUPERIOR,object,134593,0,0.00
CÓDIGO ÓRGÃO,int64,134593,0,0.00
NOME ÓRGÃO,object,134593,0,0.00
CÓDIGO UNIDADE GESTORA,int64,134593,0,0.00
NOME UNIDADE GESTORA,object,134593,0,0.00
CATEGORIA ECONÔMICA,object,134593,0,0.00
ORIGEM RECEITA,object,134593,0,0.00
ESPÉCIE RECEITA,object,134593,0,0.00
DETALHAMENTO,object,134593,0,0.00


Hay 123 valores nulos en la columna de `DATA LANÇAMENTO`, la de la fecha en la que se registró el ingreso.

In [85]:
numero_duplicados_df_2021 = datos_2021.duplicated().sum()
numero_duplicados_df_2021

np.int64(0)